### Baby Boomers Through Time
#### Demographics Report     

**Name:**      Veronica Alejandro

**UT EID:**      vaa678

#### Homework 4

##### Creating Interactive Charts to Visualize Population Shifts over Time with Altair

Baby boomers (often shortened to boomers) are the demographic cohort following the Silent Generation and preceding Generation X. The generation is generally defined as people born from 1946 to 1964, during the post–World War II baby boom. The term is also used outside the United States but the dates, the demographic context and the cultural identifiers may vary. The baby boom has been described variously as a "shockwave"and as "the pig in the python." Baby boomers are often parents of late Gen Xers and Millennials. [from wikipedia](https://en.wikipedia.org/wiki/Baby_boomers).

Let us explore this "shockwave" by examining the US Census data available via the vega datasets package.  We'll start by doing some data engineering to add a column in our population data to denote generational membership, then we will juxtapose the sex distribution of the population using a brush and linking technique we studied in the lab.  Finally we will add a slider to animate the transition through time. 

In [1]:
# Import the necessary libraries and data
import altair as alt
import pandas as pd
import numpy as np

df_pop = pd.read_json('population.json')

In [2]:
df_pop.head()

,year,age,sex,people
0,1850,0,1,1483789
1,1850,0,2,1450376
2,1850,5,1,1411067
3,1850,5,2,1359668
4,1850,10,1,1260099


In [3]:
df_pop.shape

(570, 4)

In [4]:
df_pop.describe()

,year,age,sex,people
count,570.000000,570.000000,570.000000,5.700000e+02
mean,1927.333333,45.000000,1.500000,3.428937e+06
std,46.726717,27.410182,0.500439,3.101098e+06
min,1850.000000,0.000000,1.000000,5.259000e+03
25%,1880.000000,20.000000,1.000000,6.742840e+05
50%,1930.000000,45.000000,1.500000,2.548015e+06
75%,1970.000000,70.000000,2.000000,5.466410e+06
max,2000.000000,90.000000,2.000000,1.163565e+07


In [5]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   year    570 non-null    int64
 1   age     570 non-null    int64
 2   sex     570 non-null    int64
 3   people  570 non-null    int64
dtypes: int64(4)
memory usage: 17.9 KB


##### Q1 - Add in the "Boomer" label

As we can see from inspecting the dataframe, our data only gives us information for: 
  - year
  - age
  - sex
  - people
  
But, we want to be able to highlight just the people born between 1946 - 1964 as a separate group.  To accomplish this, we want to create a new categorical attribute - `Generation`

Using pandas data manipulation techniques, add a new column to `df_pop` named `Generation` that either has the value `Baby Boomer` or `Other`. 

In [6]:
# year - age = yrborn
df_pop = df_pop.assign(Born = lambda x: (x['year'] - x['age']))
df_pop['Generation'] = ['Baby Boomer' if 1946 <= x <= 1964 else 'Other' for x in df_pop['Born']]
df_pop = df_pop.drop(['Born'], axis = 1)
df_pop

,year,age,sex,people,Generation
0,1850,0,1,1483789,Other
1,1850,0,2,1450376,Other
2,1850,5,1,1411067,Other
3,1850,5,2,1359668,Other
4,1850,10,1,1260099,Other
...,...,...,...,...,...
565,2000,80,2,3221898,Other
566,2000,85,1,970357,Other
567,2000,85,2,1981156,Other
568,2000,90,1,336303,Other


##### Q2 - Change the encoding for `sex`

As in our lab in class, the sex is "Male" is encoded as the number `1` and the sex for Female is encoded as `2`.  Modify the dataframe  `df_pop` to replace the encoding with the string so when we create our plots this will automatically have the legend come out correctly (note, you can map numbers to labels in Altair as well).  

In [7]:
df_pop['sex'] = df_pop['sex'].apply(str)

In [8]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        570 non-null    int64 
 1   age         570 non-null    int64 
 2   sex         570 non-null    object
 3   people      570 non-null    int64 
 4   Generation  570 non-null    object
dtypes: int64(3), object(2)
memory usage: 22.4+ KB


In [9]:
df_pop.head()

,year,age,sex,people,Generation
0,1850,0,1,1483789,Other
1,1850,0,2,1450376,Other
2,1850,5,1,1411067,Other
3,1850,5,2,1359668,Other
4,1850,10,1,1260099,Other


##### Q3 Juxtapose Bar Charts Horizontally 

Create a bar chart of the population distribution in the year 1960, and horizontally juxtapose the bar chart of the population distribution for the year 1990. Plot the total number of people (ignoring the `sex` attribute).

Note - You can slice the data to a given year before you pass it to Altair using pandas. (you can also do this in Altair with filters).

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers "#F4D03F" (gold) for other.

Fix the y axis so it is equal in both plots.

In [10]:
yr60 = df_pop[(df_pop['year'] == 1960)]
yr90 = df_pop[df_pop['year'] == 1990]

sixty = alt.Chart(yr60).mark_bar().encode(
    y = alt.Y('sum(people)', scale = alt.Scale(domain = [0, 25000000])),
    x = alt.X('age', scale = alt.Scale(domain = [0, 100])), 
    color = alt.Color('Generation', scale = alt.Scale(range = ['#7D3C98','#F4D03F']))
)


ninety = alt.Chart(yr90).mark_bar().encode(
    y = alt.Y('sum(people)', scale = alt.Scale(domain = [0, 25000000])),
    x = alt.X('age', scale = alt.Scale(domain = [0, 100])), 
    color = alt.Color('Generation', scale = alt.Scale(range = ['#7D3C98','#F4D03F']))
)

sixty | ninety

alt.HConcatChart(...)

##### Q5 - Show the Population Change Over Time with a Slider

Now, we have a snapshot of 2 different years next to each other, but what about creating a crude animation by controlling the the year displayed with a slider?

Create a slider using [this example](https://altair-viz.github.io/gallery/us_population_over_time.html) to help guide you.  Our plot will look similar, except we have not split our bar chart up by `sex` yet. Name the slider 'Select Year:' (this in controlled in `binding_range`, and not in the `selection_single` parameters).  

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers "#F4D03F" (gold) for other.

Start the slider at 1900.

In [11]:
slider = alt.binding_range(min = 1900, max = 2000, step = 10, name = 'Select Year:')
selector = alt.selection_single(fields = ['year'], bind = slider)

poptime = alt.Chart(df_pop).mark_bar().encode(
    y = alt.Y('sum(people)'),
    x = alt.X('age'), 
    color = alt.Color('Generation', scale = alt.Scale(domain = ['Baby Boomer', 'Other'], range = ['#7D3C98','#F4D03F']))
).add_selection(
    selector
).transform_filter(
    selector
)

poptime

alt.Chart(...)

##### Q6 - Linking

Let us take a closer look at just the year 2000 data, and find what the distribution of sex is for each individual age grouping.  Plot the distribution of ages as a bar chart for just the year 2000, and link a histogram that will plot the distribution of sex for the current selection.  It should default to no age group selected.  The histogram for the sex distribution should appear below the year 2000 data (vertically concatenated). When a bar on the top chart is selected, indicate its selection by turning the other bars light gray.  The histogram of the sex distribution below it should be a horizontal bar chart. 

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers "#F4D03F" (gold) for other.

In [12]:
yr00 = df_pop[df_pop['year'] == 2000]

interval = alt.selection_interval()

aughts = alt.Chart(yr00).mark_bar().encode(
    x = 'age', y = 'sum(people)', 
    color = alt.condition (interval, 'Generation', alt.value('lightgray'))
).add_selection(
    interval
)

sex = alt.Chart(yr00).mark_bar().encode(
    x = 'sum(people)', y = 'sex', color = alt.Color('Generation', scale = alt.Scale(domain = ['Baby Boomer', 'Other'], range = ['#7D3C98','#F4D03F']))
).transform_filter(
    interval
)

link = aughts & sex
link

alt.VConcatChart(...)

##### Q7 - Combine Q5 and Q6 to One Chart

In question 6, we linked the distribution of sex to the age selection for just the year 2000.  Let us visualize all the data by incorporating the year selection slider from question 5 so that you can select which year of data you are viewing. Retain the ability to just select one age group for the sex distribution, and default to no age group selected.

Add a tooltip so you can see exactly how many people are in the age range for the top "Distribution of Ages for the Selected Year" histogram. 

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers "#F4D03F" (gold) for other.

In [13]:
interval = alt.selection_interval()

all = alt.Chart(df_pop).mark_bar().encode(
    x = 'age', y = 'sum(people)', 
    color = alt.condition (interval, 'Generation', alt.value('lightgray'))
).add_selection(
    interval
)

sex = alt.Chart(df_pop).mark_bar().encode(
    x = 'sum(people)', y = 'sex', color = alt.Color('Generation', scale = alt.Scale(domain = ['Baby Boomer', 'Other'], range = ['#7D3C98','#F4D03F']))
).transform_filter(
    interval
)

link = all & sex

link.add_selection(
    selector
).transform_filter(
    selector
)

alt.VConcatChart(...)